# Clustering of transcripts upstream of TSS

Work on clustering transcripts using their conservation score 1 kb upstream of TSS:
- slice
- intersect w/conservation scores
- 10 bp bins
- avg score w/in bins
- table
- standardize
- cluster! 

In [2]:
import gffutils
from gffutils import pybedtools_integration
import pybedtools
from pybedtools.featurefuncs import gff2bed
import pandas as pd
import seaborn as sb
import statsmodels
import statsmodels.api as sm
%matplotlib inline

/data/bergeric/miniconda3/envs/s2rnai/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Import database and define transcripts, genes, tsses, etc. 

In [37]:
db = gffutils.FeatureDB('/data/LCDB/lcdb-references/dmel/r6-11/gtf/dmel_r6-11.gtf.db')

/data/bergeric/miniconda3/envs/s2rnai/lib/python3.5/site-packages/gffutils/interface.py:161: UserWarning: It appears that this database has not had the ANALYZE sqlite3 command run on it. Doing so can dramatically speed up queries, and is done by default for databases created with gffutils >0.8.7.1 (this database was created with version 0.8.7.1) Consider calling the analyze() method of this object.
  "method of this object." % self.version)


In [38]:
transcripts = pybedtools_integration.to_bedtool(db.features_of_type('transcript')).saveas()

In [39]:
genes = pybedtools_integration.to_bedtool(db.features_of_type('gene')).sort().merge().saveas()

In [40]:
tss = pybedtools_integration.tsses(db, merge_overlapping=False).saveas('../../output/tsses.gff')

In [41]:
slop = tss.slop(l=1000, r=0, s=True, genome='dm6').saveas('../../output/another_slop.bed')

Get intersections TSS w/exons, TSS w/introns, TSS w/intergenic:

In [11]:
TSS_exons = tss.intersect('../../output/dm6_exons.bed').saveas().to_dataframe()

In [21]:
introns = pd.read_table('../../output/dmel-introns-r6.11.gff', header=None)

In [27]:
introns[0] = [ 'chr'+x for x in introns[0] ] 

In [35]:
tss_introns = tss.intersect(pybedtools.BedTool.from_dataframe(introns), wa=True).saveas().to_dataframe()

In [16]:
TSS_intergenic = tss.intersect('../../output/intergenic.bed').saveas().to_dataframe()

Intersect upstream of TSS w/conservation scores:

(If I try to do here will fail, probably because I don't have enough memory? so doing it on local machine)

In [ ]:
#wig2bed < dm6.27way.phastCons.wigFix > dm6_phastcons.bed
phastcons = pybedtools.BedTool('../../output/dm6_phastcons.bed').saveas()

In [42]:
#import intersect file: (really big, splitting by chrom)
intersect_2L = pd.read_table('../../output/2L_intersect', header=None)

want to break up into 10 bp bins, with average conservation score. In order to do that, 
### iterate by TSS:  

- read in another slop line by line
- for each line, make windows, then intersect with phastcons, then take that and agg over scores
- intersect back with another_slop to get gene/transcript info? or could take that out in parser maybe? 

In [7]:
tsses = pd.read_table('../../output/motif/another_slop.bed', header=None)

### 2L

In [8]:
tsses_2L = tsses[tsses[0] == 'chr2L']

In [9]:
tsses_2L[8] = [ x.split()[3].strip(';').strip('"') for x in list(tsses_2L[8]) ]


/data/bergeric/miniconda3/envs/s2rnai/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [10]:
tsses_2L = tsses_2L.groupby([0,3,4])[8].apply(lambda x:'|'.join(x)).reset_index()

In [11]:
tsses_2L.head()

,0,3,4,8
0,chr2L,6529,7529,FBtr0300689|FBtr0300690|FBtr0330654
1,chr2L,18570,19570,FBtr0078170|FBtr0078171
2,chr2L,20952,21952,FBtr0309810
3,chr2L,21376,22376,FBtr0078168|FBtr0078166|FBtr0078167|FBtr007816...
4,chr2L,25155,26155,FBtr0113008


In [13]:
phastcons = pybedtools.BedTool('../../output/motif/2L_phastcons.bed').saveas()

In [29]:
done = pd.read_csv('../../output/done_from_today', header=None)

In [58]:
done_vals = []
for x, y in done.iterrows():
    if list(y) not in done_vals:
        done_vals.append(list(y))

In [59]:
for name, group in tsses_2L.groupby([3,4]):
    if [str(name), str(group)] not in done_vals:
        print([str(name), str(group)])
        break

['(19079028, 19080028)', '          0         3         4            8\n3700  chr2L  19079028  19080028  FBtr0081171']


In [60]:
def make_bins(chrom,a,b,n):
    bins = pd.cut([a,b], n, retbins=True, include_lowest=True)[1]
    collect = []
    for i in range(n):
        collect.append([chrom, int(list(bins)[i:i+2][0]), int(list(bins)[i:i+2][1])])
    return collect

errors=[]
for name, group in tsses_2L.groupby([3,4]):
    if [str(name), str(group)] not in done_vals:
        concat=[]
        done_vals.append([str(name), str(group)])
        bins = make_bins(list(group[0])[0], name[0], name[1], 100)
        df = pd.DataFrame(bins)
        intersect = pybedtools.BedTool.from_dataframe(df).intersect(phastcons, wo=True)
        try:
            intersect = intersect.to_dataframe()
            agg = intersect.groupby(['chrom', 'start', 'end']).agg({'thickEnd':'mean'}).reset_index()
            with_name = agg.copy()
            with_name['name'] = list(group[8])[0]
            concat.append(with_name)
            final = pd.concat(concat)
            final = final[['chrom','start','end','name','thickEnd']]
            final.to_csv('../../output/tss_windows_wphastcons_3', sep='\t', header=None, index=False, mode='a')
        except: 
            errors.append(name)
        pd.DataFrame(done_vals).to_csv('../../output/done_herewegoagain', header=None, index=False, mode='a')

In [61]:
#this actually finished!! So need to go through and make sure everything looks ok but leaving for now
#also haven't figured out what to do about my git branching issue :/ 
pd.read_table('../../output/tss_windows_wphastcons_3', header=None)

,0,1,2,3,4
0,chr2L,6528,6539,FBtr0300689|FBtr0300690|FBtr0330654,0.078182
1,chr2L,6539,6549,FBtr0300689|FBtr0300690|FBtr0330654,0.100600
2,chr2L,6549,6559,FBtr0300689|FBtr0300690|FBtr0330654,0.002000
3,chr2L,6559,6569,FBtr0300689|FBtr0300690|FBtr0330654,0.004700
4,chr2L,6569,6579,FBtr0300689|FBtr0300690|FBtr0330654,0.015100
5,chr2L,6579,6589,FBtr0300689|FBtr0300690|FBtr0330654,0.001200
6,chr2L,6589,6599,FBtr0300689|FBtr0300690|FBtr0330654,0.000600
7,chr2L,6599,6609,FBtr0300689|FBtr0300690|FBtr0330654,0.003400
8,chr2L,6609,6619,FBtr0300689|FBtr0300690|FBtr0330654,0.007500
9,chr2L,6619,6629,FBtr0300689|FBtr0300690|FBtr0330654,0.001500
